In [ ]:
import pandas as pd

df = pd.read_csv("train.csv")
df.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [ ]:
!pip install -U sentence-transformers scikit-learn pandas numpy joblib

In [ ]:
!pip install "scikit-learn<1.7,>=1.2" --upgrade --force-reinstall
!python -c "import sklearn; print(sklearn.__version__)"

  Using cached numpy-2.3.4-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 24.2 MB/s eta 0:00:00
Using cached numpy-2.3.4-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 9.1 MB/s eta 0:00:00
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.6.0
    Uninstalling threadpoolctl-3.6.0:
      Successfully uninstalled threadpoolctl-3.6.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.4
    Uninstalling numpy-2.3.4:
      Successfully uninstalled numpy-2.3.4
  Attempting uninstall: joblib
    Found existing installation: joblib 1.5.2
    Uninstalling joblib-1.5.2:
      Successfully uninstalled joblib-1.5.2
  A

1.6.1


## Our code starts here

In [ ]:
import sklearn, pandas, numpy, sentence_transformers
print("sklearn:", sklearn.__version__)
print("pandas:", pandas.__version__)
print("numpy:", numpy.__version__)
print("sBERT:", sentence_transformers.__version__)

from sentence_transformers import SentenceTransformer
m = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device="cpu")
emb = m.encode(["Cyclone warning near Odisha coast", "Cute cat video"], normalize_embeddings=True)
print(emb.shape)

sklearn: 1.6.1
pandas: 2.3.3
numpy: 2.3.4
sBERT: 5.1.2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(2, 384)


In [ ]:
import sys, subprocess
print("Kernel Python:", sys.executable)
subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "sentence-transformers"])

Kernel Python: /usr/bin/python3


0

In [ ]:
import sentence_transformers, sklearn, pandas, numpy
print("sentence-transformers:", sentence_transformers.__version__)
print("sklearn:", sklearn.__version__, "pandas:", pandas.__version__, "numpy:", numpy.__version__)
from sentence_transformers import SentenceTransformer
m = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device="cpu")
print("Embedding shape:", m.encode(["ok"]).shape)

sentence-transformers: 5.1.2
sklearn: 1.6.1 pandas: 2.3.3 numpy: 2.3.4
Embedding shape: (1, 384)


In [ ]:
import os, re, numpy as np, pandas as pd, joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, precision_recall_fscore_support
from sentence_transformers import SentenceTransformer

# ========= Config =========
DATA_CSV = "train.csv"  # adjust path if needed
TEXT_COL = "text"
LABEL_COL = "target"   # 1=relevant, 0=irrelevant (from Kaggle disaster tweets)
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
ARTIFACT_DIR = "artifacts/slm_relevance"   # where to save models/embeddings
os.makedirs(ARTIFACT_DIR, exist_ok=True)

# ========= Load & clean =========
df = pd.read_csv(DATA_CSV)
df = df[[TEXT_COL, LABEL_COL]].dropna()
df[TEXT_COL] = df[TEXT_COL].astype(str)

def clean_text(s: str) -> str:
    s = re.sub(r"http\S+|www\.\S+", " ", s)          # links
    s = re.sub(r"#(\w+)", r"\1", s)                  # hashtags -> word
    s = re.sub(r"@\w+", " ", s)                      # mentions
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["clean"] = df[TEXT_COL].apply(clean_text)

# stratified split
train_df, val_df = train_test_split(
    df, test_size=0.15, random_state=42, stratify=df[LABEL_COL]
)

print("Train size:", len(train_df), "Val size:", len(val_df))
print("Label dist (train):\n", train_df[LABEL_COL].value_counts(normalize=True))

# ========= Encode with SentenceTransformer (CPU) =========
st_model = SentenceTransformer(MODEL_NAME, device="cpu")

def embed(texts, batch_size=256):
    return st_model.encode(
        texts, batch_size=batch_size, convert_to_numpy=True, show_progress_bar=True, normalize_embeddings=True
    )

X_train = embed(train_df["clean"].tolist())
y_train = train_df[LABEL_COL].values.astype(int)

X_val = embed(val_df["clean"].tolist())
y_val = val_df[LABEL_COL].values.astype(int)

np.save(os.path.join(ARTIFACT_DIR, "X_train.npy"), X_train)
np.save(os.path.join(ARTIFACT_DIR, "y_train.npy"), y_train)
np.save(os.path.join(ARTIFACT_DIR, "X_val.npy"),   X_val)
np.save(os.path.join(ARTIFACT_DIR, "y_val.npy"),   y_val)

# ========= Train a lightweight classifier =========
# class_weight='balanced' handles imbalance nicely
clf = LogisticRegression(
    max_iter=2000,
    class_weight="balanced",
    n_jobs=1,
    C=2.0,            # a bit more capacity than default
    solver="lbfgs"
)
clf.fit(X_train, y_train)

# ========= Evaluate =========
pred = clf.predict(X_val)
proba = clf.predict_proba(X_val)[:, 1]

print(classification_report(y_val, pred, digits=4))
p, r, f1, _ = precision_recall_fscore_support(y_val, pred, average="binary")
print({"precision": p, "recall": r, "f1": f1})

# ========= Save artifacts =========
joblib.dump(clf, os.path.join(ARTIFACT_DIR, "lr_relevance.joblib"))
st_model.save(os.path.join(ARTIFACT_DIR, "sbert_model"))  # saves config + modules

print("Saved to:", ARTIFACT_DIR)

Train size: 6471 Val size: 1142
Label dist (train):
 target
0    0.570391
1    0.429609
Name: proportion, dtype: float64


Batches:   0%|          | 0/26 [00:00<?, ?it/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0     0.8465    0.8387    0.8426       651
           1     0.7887    0.7984    0.7935       491

    accuracy                         0.8214      1142
   macro avg     0.8176    0.8185    0.8181      1142
weighted avg     0.8217    0.8214    0.8215      1142

{'precision': 0.7887323943661971, 'recall': 0.7983706720977597, 'f1': 0.7935222672064778}
Saved to: artifacts/slm_relevance


In [ ]:

import numpy as np, joblib
from sentence_transformers import SentenceTransformer

ARTIFACT_DIR = "artifacts/slm_relevance"
SBERT_PATH   = f"{ARTIFACT_DIR}/sbert_model"
LR_PATH      = f"{ARTIFACT_DIR}/lr_relevance.joblib"

_sbert = SentenceTransformer(SBERT_PATH, device="cpu")
_lr    = joblib.load(LR_PATH)

def predict_relevance(text: str, threshold: float = 0.5):
    if not text or not text.strip():
        return False, 0.0
    emb = _sbert.encode([text], convert_to_numpy=True, normalize_embeddings=True)
    p = float(_lr.predict_proba(emb)[0, 1])
    return (p >= threshold, p)

In [ ]:

print(predict_relevance("love of my girlfreind flodded into my heart that caused me to believe I am in heaven "))
print(predict_relevance("becouse of earhquick 12 peaople died in rourkela "))
print(predict_relevance("I became so horny that I fucked my cuh so hard and ejaculated a huge tsunami of cum which destroyed here whole family devastatingly"))


(False, 0.09232666498762844)
(True, 0.9178836518200003)
(False, 0.2962920973074937)


## hyperparameter tuning for logistic regression

In [ ]:
import numpy as np
import os
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# --- Config ---
ARTIFACT_DIR = "artifacts/slm_relevance"

# --- Load Existing Embeddings ---
print("Loading pre-computed embeddings...")
X_train = np.load(os.path.join(ARTIFACT_DIR, "X_train.npy"))
y_train = np.load(os.path.join(ARTIFACT_DIR, "y_train.npy"))
X_val = np.load(os.path.join(ARTIFACT_DIR, "X_val.npy"))
y_val = np.load(os.path.join(ARTIFACT_DIR, "y_val.npy"))
print(f"Loaded {len(X_train)} train and {len(X_val)} val samples.")

# --- Setup GridSearchCV ---
print("Starting GridSearchCV for LogisticRegression...")

# Define the model
lr = LogisticRegression(max_iter=2000, class_weight="balanced", solver="lbfgs")

# Define the grid of parameters to search
# We'll search different values for 'C' (regularization strength)
# Your original code used C=2.0
param_grid = {
    'C': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
}

# Set up the grid search with 5-fold cross-validation
# 'f1' scoring is good for this balanced-but-not-perfectly-so dataset
grid_search = GridSearchCV(estimator=lr, param_grid=param_grid, cv=5, scoring='f1', n_jobs=-1, verbose=1)

# Run the grid search on the training data
grid_search.fit(X_train, y_train)

# --- Evaluate Best Model ---
print(f"\nBest parameters found: {grid_search.best_params_}")
print(f"Best cross-validation F1-score: {grid_search.best_score_:.4f}")

# Get the best model found by the search
best_lr = grid_search.best_estimator_

# Evaluate the best model on the held-out validation set
print("\n--- Report on Validation Set (using best model) ---")
pred_val = best_lr.predict(X_val)
print(classification_report(y_val, pred_val, digits=4))

# --- Save the new, tuned model ---
tuned_model_path = os.path.join(ARTIFACT_DIR, "lr_relevance_tuned.joblib")
joblib.dump(best_lr, tuned_model_path)
print(f"Saved tuned LogisticRegression model to: {tuned_model_path}")


Loading pre-computed embeddings...
Loaded 6471 train and 1142 val samples.
Starting GridSearchCV for LogisticRegression...
Fitting 5 folds for each of 6 candidates, totalling 30 fits

Best parameters found: {'C': 5.0}
Best cross-validation F1-score: 0.7681

--- Report on Validation Set (using best model) ---
              precision    recall  f1-score   support

           0     0.8426    0.8387    0.8406       651
           1     0.7874    0.7923    0.7898       491

    accuracy                         0.8187      1142
   macro avg     0.8150    0.8155    0.8152      1142
weighted avg     0.8189    0.8187    0.8188      1142

Saved tuned LogisticRegression model to: artifacts/slm_relevance/lr_relevance_tuned.joblib


## using lightgbm with hyperparameter tuning

In [ ]:
import numpy as np
import os
import joblib
try:
    from lightgbm import LGBMClassifier
except ImportError:
    print("LightGBM not found. Installing...")
    # This magic command might only work in notebooks.
    # If not in a notebook, run: pip install lightgbm
    !pip install lightgbm
    from lightgbm import LGBMClassifier

from sklearn.metrics import classification_report

# --- Config ---
ARTIFACT_DIR = "artifacts/slm_relevance"

# --- Load Existing Embeddings ---
print("Loading pre-computed embeddings...")
X_train = np.load(os.path.join(ARTIFACT_DIR, "X_train.npy"))
y_train = np.load(os.path.join(ARTIFACT_DIR, "y_train.npy"))
X_val = np.load(os.path.join(ARTIFACT_DIR, "X_val.npy"))
y_val = np.load(os.path.join(ARTIFACT_DIR, "y_val.npy"))
print(f"Loaded {len(X_train)} train and {len(X_val)} val samples.")

# --- Train LGBMClassifier ---
print("Training LGBMClassifier...")

# 'class_weight="balanced"' is also available here
lgbm_clf = LGBMClassifier(
    n_estimators=200,       # Number of trees
    learning_rate=0.05,
    class_weight="balanced",
    n_jobs=-1,
    random_state=42
)

lgbm_clf.fit(X_train, y_train)

# --- Evaluate LGBM Model ---
print("\n--- Report on Validation Set (LGBM) ---")
pred_val = lgbm_clf.predict(X_val)
print(classification_report(y_val, pred_val, digits=4))

# --- Save the new LGBM model ---
lgbm_model_path = os.path.join(ARTIFACT_DIR, "lgbm_relevance.joblib")
joblib.dump(lgbm_clf, lgbm_model_path)
print(f"Saved LGBM model to: {lgbm_model_path}")


Loading pre-computed embeddings...
Loaded 6471 train and 1142 val samples.
Training LGBMClassifier...
[LightGBM] [Info] Number of positive: 2780, number of negative: 3691
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.060522 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 97920
[LightGBM] [Info] Number of data points in the train set: 6471, number of used features: 384
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000

--- Report on Validation Set (LGBM) ---
              precision    recall  f1-score   support

           0     0.8267    0.8571    0.8416       651
           1     0.8009    0.7617    0.7808       491

    accuracy                         0.8161      1142
   macro avg     0.8138    0.8094    0.8112      1142
weighted avg     0.8156    0.8161    0.8155      1142

Saved LGBM model to: artifacts/slm_re

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
import os, re, numpy as np, pandas as pd, joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sentence_transformers import SentenceTransformer

# ========= Config (NEW) =========
DATA_CSV = "train.csv"  # Make sure this path is correct
TEXT_COL = "text"
LABEL_COL = "target"

# --- Use a more powerful model ---
MODEL_NAME_NEW = "sentence-transformers/all-mpnet-base-v2"
EMBEDDING_DIM = 768 # This model outputs 768-dim vectors, not 384

# --- Save to a new directory to avoid overwriting your original artifacts ---
ARTIFACT_DIR_NEW = "artifacts/slm_relevance_mpnet"
os.makedirs(ARTIFACT_DIR_NEW, exist_ok=True)

# ========= Load & clean (Same as before) =========
df = pd.read_csv(DATA_CSV)
df = df[[TEXT_COL, LABEL_COL]].dropna()
df[TEXT_COL] = df[TEXT_COL].astype(str)

def clean_text(s: str) -> str:
    s = re.sub(r"http\S+|www\.\S+", " ", s)          # links
    s = re.sub(r"#(\w+)", r"\1", s)                  # hashtags -> word
    s = re.sub(r"@\w+", " ", s)                      # mentions
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["clean"] = df[TEXT_COL].apply(clean_text)

# stratified split (Same as before)
train_df, val_df = train_test_split(
    df, test_size=0.15, random_state=42, stratify=df[LABEL_COL]
)

print(f"Train size: {len(train_df)}, Val size: {len(val_df)}")
print(f"Using new model: {MODEL_NAME_NEW}")

# ========= Encode with NEW SentenceTransformer =========
st_model_new = SentenceTransformer(MODEL_NAME_NEW, device="cpu")

def embed(texts, batch_size=128): # Smaller batch size for larger model
    return st_model_new.encode(
        texts, batch_size=batch_size, convert_to_numpy=True, show_progress_bar=True, normalize_embeddings=True
    )

print("\nEmbedding training data (this may take longer)...")
X_train_new = embed(train_df["clean"].tolist())
y_train = train_df[LABEL_COL].values.astype(int)

print("Embedding validation data...")
X_val_new = embed(val_df["clean"].tolist())
y_val = val_df[LABEL_COL].values.astype(int)

print(f"New embedding shape: {X_train_new.shape}")

# --- Save new artifacts ---
np.save(os.path.join(ARTIFACT_DIR_NEW, "X_train.npy"), X_train_new)
np.save(os.path.join(ARTIFACT_DIR_NEW, "y_train.npy"), y_train)
np.save(os.path.join(ARTIFACT_DIR_NEW, "X_val.npy"),   X_val_new)
np.save(os.path.join(ARTIFACT_DIR_NEW, "y_val.npy"),   y_val)

# ========= Train classifier on NEW embeddings =========
print("\nTraining LogisticRegression on new embeddings...")
clf_new = LogisticRegression(
    max_iter=2000,
    class_weight="balanced",
    n_jobs=1,
    C=2.0, # You could also run GridSearchCV on this new model!
    solver="lbfgs"
)
clf_new.fit(X_train_new, y_train)

# ========= Evaluate =========
print("\n--- Report on Validation Set (new model) ---")
pred = clf_new.predict(X_val_new)
print(classification_report(y_val, pred, digits=4))

p, r, f1, _ = precision_recall_fscore_support(y_val, pred, average="binary")
print(f"New F1 Score (binary): {f1:.4f}")

# ========= Save NEW artifacts =========
joblib.dump(clf_new, os.path.join(ARTIFACT_DIR_NEW, "lr_relevance.joblib"))
st_model_new.save(os.path.join(ARTIFACT_DIR_NEW, "sbert_model"))

print(f"\nSaved new artifacts to: {ARTIFACT_DIR_NEW}")


Train size: 6471, Val size: 1142
Using new model: sentence-transformers/all-mpnet-base-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...


Embedding training data (this may take longer)...


Batches:   0%|          | 0/51 [00:00<?, ?it/s]

Embedding validation data...


Batches:   0%|          | 0/9 [00:00<?, ?it/s]

New embedding shape: (6471, 768)

Training LogisticRegression on new embeddings...

--- Report on Validation Set (new model) ---
              precision    recall  f1-score   support

           0     0.8535    0.8326    0.8429       651
           1     0.7850    0.8106    0.7976       491

    accuracy                         0.8231      1142
   macro avg     0.8193    0.8216    0.8203      1142
weighted avg     0.8241    0.8231    0.8234      1142

New F1 Score (binary): 0.7976

Saved new artifacts to: artifacts/slm_relevance_mpnet
